# Test drift scan code
- this notebook can be used for debugging and testing 

In [1]:
from glob import glob
import os

from argparse import ArgumentParser, RawTextHelpFormatter
from astropy.coordinates import SkyCoord, FK5
from astropy.io import fits
from astropy.time import Time
from astropy.table import Table
import astropy.units as u
from astropy.wcs import WCS
import numpy as np
from scipy import interpolate
import time
from bisect import bisect_left
from astropy.io import ascii

from drift_analysis.modules.telescope_params import westerbork

In [2]:
def task_id2equinox(task_id):

    # Automatically take the date of the observaitons from the task_id to calculate apparent coordinates of calibrator
    year = 2000 + int(str(task_id)[0:2])
    month = str(task_id)[2:4]
    day = str(task_id)[4:6]
    equinox = Time('{}-{}-{}'.format(year, month, day))

    return equinox.decimalyear

def take_closest(myList, myNumber):
    """
    Assumes myList is sorted. Returns closest value to myNumber.

    If two numbers are equally close, return the smallest number.
    """
    pos = bisect_left(myList, myNumber)
    myList.sort()
    if pos == 0:
        return myList[0]
    if pos == len(myList):
        return myList[-1]
    before = myList[pos - 1]
    after = myList[pos]
    if after - myNumber < myNumber - before:
       return pos - 1
    else:
       return pos

def make_gifs(root):

    os.system('convert -delay 50 {}*db0_reconstructed.png {}all_beams0.gif'.format(root, root))
    os.system('convert -delay 50 {}*_difference.png {}diff_xx-yy.gif'.format(root, root))

    return

In [3]:
#start = time.time()

beam_range = [0,39]
beams = range(int(beam_range[0]), int(beam_range[1])+1)
freqchunks = 10
date = '201028'

#print(beams)

basedir = '/tank/apertif/driftscans/'

#with open('drift_analysis/task_id_lists/task_ids_{}.txt'.format(date)) as f:
with open('drift_analysis/task_id_lists/task_ids_{}_casa.txt'.format(date)) as f:
    task_id = f.read().splitlines()	

np.warnings.filterwarnings('ignore')

radec = ascii.read('/home/denes/pattern39+1.txt')


# Find calibrator position
calib_name = 'Cas A'
calib = SkyCoord.from_name('Cas A')

cell_size = 100. / 3600.
l = len(task_id)

# Make a list of data tables (the csv tables with the auto correlations).
datafiles, posfiles = [], []

for i in range(len(task_id)):
    datafiles.append('{}{}/{}_exported_data_frequency_split.csv'.format(basedir, task_id[i], task_id[i]))
    posfiles.append('{}{}/{}_hadec.csv'.format(basedir, task_id[i], task_id[i]))

#datafiles.sort()
#posfiles.sort()
#print(datafiles)

# Put calibrator into apparent coordinates (because that is what the telescope observes it in.)
test = calib.transform_to('fk5')
calibnow = test.transform_to(FK5(equinox='J{}'.format(task_id2equinox(task_id[0]))))

# Read data from tables
data_tab, hadec_tab = [], []
print("\nReading in all the data...")
for file, pos in zip(datafiles, posfiles):
    data_tab.append(Table.read(file, format='csv'))  # list of tables
    hadec_tab.append(Table.read(pos, format='csv'))  # list of tables




Reading in all the data...


In [4]:
beams = range(0,40)
#calib_name = 'Cyg A'
calib_name = 'Cas A'


In [9]:
print("Making beam maps: ")
for beam in beams:
    print(beam)

    for f in range(freqchunks):
        x, y, z_xx, z_yy = [], [], [], []
        decs = []

        for data, hadec, i in zip(data_tab, hadec_tab, range(len(data_tab))):
            calibnow = test.transform_to(FK5(equinox='J{}'.format(task_id2equinox(task_id[i]))))

            hadec_start = SkyCoord(ra=hadec['ha'], dec=hadec['dec'], unit=(u.rad, u.rad))
            time_mjd = Time(data['time'] / (3600 * 24), format='mjd')
            time_mjd.delta_ut1_utc = 0  # extra line to compensate for missing icrs tables
            lst = time_mjd.sidereal_time('apparent', westerbork().lon)

            HAcal = lst - calibnow.ra  # in sky coords
            dHAsky = HAcal - hadec_start[beam].ra + (24 * u.hourangle)  # in sky coords in hours
            dHAsky.wrap_at('180d', inplace=True)
            dHAphys = dHAsky * np.cos(hadec_start[beam].dec.deg * u.deg)  # physical offset in hours

            x = np.append(x, dHAphys.deg)
            y = np.append(y, np.full(len(dHAphys.deg), hadec_start[beam].dec.deg))  # the drift scan sampling is not uniform on the top of the field, linspace squashes the top beams 
            z_xx = np.append(z_xx, data['auto_corr_beam_{}_freq_{}_xx'.format(beam, f)] - np.median(
                data['auto_corr_beam_{}_freq_{}_xx'.format(beam, f)]))
            z_yy = np.append(z_yy, data['auto_corr_beam_{}_freq_{}_yy'.format(beam, f)] - np.median(
                data['auto_corr_beam_{}_freq_{}_yy'.format(beam, f)]))

        # Create the 2D plane, do a cubic interpolation, and append it to the cube.
        tx = np.arange(min(x), max(x), cell_size)
        ty = np.arange(min(y), max(y), cell_size)
        XI, YI = np.meshgrid(tx, ty)
        #print(len(ty))
        #gridcubx = np.flipud(interpolate.griddata((x, y), z_xx, (XI, YI), method='cubic'))  # median already subtracted
        #gridcuby = np.flipud(interpolate.griddata((x, y), z_yy, (XI, YI), method='cubic'))
        gridcubx = interpolate.griddata((x, y), z_xx, (XI, YI), method='cubic')  # median already subtracted
        gridcuby = interpolate.griddata((x, y), z_yy, (XI, YI), method='cubic')


        # Find the reference pixel at the apparent coordinates of the calibrator
        #ref_pixy = (calibnow.dec.deg - y[0]) / cell_size + 1  # FITS indexed from 1
        calibnow = test.transform_to(FK5(equinox='J{}'.format(task_id2equinox(task_id[0]))))
        ref_pixx = (-min(x)) / cell_size + 1                        # FITS indexed from 1
        ref_pixy = (calibnow.dec.deg - min(y)) / cell_size + 1   
        ref_pixz = 1                                                # FITS indexed from 1

        # Find the peak of the primary beam to normalize
        norm_xx = np.max(gridcubx[int(ref_pixy) - 3:int(ref_pixy) + 4, int(ref_pixx) - 3:int(ref_pixx) + 4])
        norm_yy = np.max(gridcuby[int(ref_pixy) - 3:int(ref_pixy) + 4, int(ref_pixx) - 3:int(ref_pixx) + 4])

        # Create 3D array with proper size for given scan set to save data as a cube
        if f == 0:
            cube_xx = np.zeros((freqchunks, gridcubx.shape[0], gridcubx.shape[1]))
            cube_yy = np.zeros((freqchunks, gridcuby.shape[0], gridcuby.shape[1]))
            db_xx = np.zeros((freqchunks, gridcubx.shape[0], gridcubx.shape[1]))
            db_yy = np.zeros((freqchunks, gridcuby.shape[0], gridcuby.shape[1]))

        cube_xx[f, :, :] = gridcubx/norm_xx
        cube_yy[f, :, :] = gridcuby/norm_yy

        # Convert to decibels
        db_xx[f, :, :] = np.log10(gridcubx/norm_xx) * 10.
        db_yy[f, :, :] = np.log10(gridcuby/norm_yy) * 10.

    stokesI = np.sqrt(0.5 * cube_yy**2 + 0.5 * cube_xx**2)
    squint = cube_xx - cube_yy

    wcs = WCS(naxis=3)
    wcs.wcs.cdelt = np.array([-cell_size, cell_size, 12.207e3*1500])
    wcs.wcs.ctype = ['RA---TAN', 'DEC--TAN', 'FREQ']
    wcs.wcs.crval = [calib.ra.to_value(u.deg), calib.dec.to_value(u.deg), 1219.609e6+(12.207e3*(500+1500/2))]
    wcs.wcs.crpix = [ref_pixx, ref_pixy, ref_pixz]
    wcs.wcs.specsys = 'TOPOCENT'
    wcs.wcs.restfrq = 1.420405752e+9
    header = wcs.to_header()

    hdux = fits.PrimaryHDU(cube_xx, header=header)
    hduy = fits.PrimaryHDU(cube_yy, header=header)
    hduI = fits.PrimaryHDU(stokesI, header=header)
    hdusq = fits.PrimaryHDU(squint, header=header)

    if not os.path.exists(basedir + 'fits_files/{}/'.format(date)):
        os.mkdir(basedir + 'fits_files/{}/'.format(date))

    # Save the FITS files
    hdux.writeto(basedir + 'fits_files/{}/{}_{}_{:02}_xx.fits'.format(date, calib_name.replace(" ", ""), date,
                                                         beam), overwrite=True)
    hduy.writeto(basedir + 'fits_files/{}/{}_{}_{:02}_yy.fits'.format(date, calib_name.replace(" ", ""), date,
                                                         beam), overwrite=True)
    hduI.writeto(basedir + 'fits_files/{}/{}_{}_{:02}_I.fits'.format(date, calib_name.replace(" ", ""), date,
                                                         beam), overwrite=True)
    hdusq.writeto(basedir + 'fits_files/{}/{}_{}_{:02}_diff.fits'.format(date, calib_name.replace(" ", ""), date,
                                                             beam), overwrite=True)

#end = time.time()
#print('Time [minutes]: ', (end - start)/60)

Making beam maps: 
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
